In [84]:
import requests
import json
import re
import pandas as pd
from itertools import chain

In [74]:
def get_assessment(url):
    fes_collection_api = "https://w3id.org/FAIR_Evaluator/collections/6/evaluate"

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    payload = {
        "resource": url,
        "executor": "mwurzb",
        "title": "comp_doi_test"
    }

    response = requests.post(fes_collection_api, headers=headers, json=payload)
    response_json = response.json()
    eval_result = json.loads(response_json.get("evaluationResult"))
    global count
    print(count)
    count = count + 1
    print("_________________________")
    return eval_result

In [66]:
def get_metricurl(url):
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    response = requests.get(url=url, headers=headers)
    return response

In [67]:
def get_metrics_and_scores(eval_result):
    eval_result_keys = eval_result.keys()
    results = []
    for k in eval_result_keys:
        metric_url = get_metricurl(k).json().get("test_of_metric")
        metric_abbrev = re.search(r'(?<=Gen2_FM_).*$', metric_url)
        metric = metric_abbrev.group()

        eval_result_val = eval_result.get(k)
        key = "http://semanticscience.org/resource/SIO_000300"
        metric_score = int(eval_result_val[0][key][0]["@value"])

        results.append(
            {
                "metric": metric,
                "score": metric_score
            }
        )
    return results

In [86]:
def get_metrics_from_result(assessment_result):
    return [item.get('metric') for item in assessment_result]

def get_scores_from_result(assessment_result):
    return [int(item.get('score')) for item in assessment_result]

# Papers

# Comparisons

## Comparisons via DOI

In [90]:
def expand_comp_assessment_result(df):
    df_exp = pd.DataFrame()
    df_exp["comparison"] = list(chain.from_iterable([[x]*22 for x in df["comparisons"]]))
    try:
        df_exp["doi"] = list(chain.from_iterable([[x]*22 for x in df["doi"]]))
    except KeyError:
        pass
    df_exp["metric"] = list(chain.from_iterable([get_metrics_from_result(assessment_result=x) for x in df["assessment_result"]]))
    df_exp["score"] = list(chain.from_iterable([get_scores_from_result(assessment_result=x) for x in df["assessment_result"]]))
    df_exp = df_exp[["comparison", "doi", "metric", "score"]]
    return df_exp

### Comparisons assessment via DOI

In [77]:
%%script false --no-raise-error
comp_doi_df = pd.read_csv("raw_data/comparison_query_result_2023-03-29.csv")
comp_doi_df = comp_doi_df[comp_doi_df["doi"].isna() == False].reset_index(drop=True)
count = 0
comp_doi_df["assessment_result"] = comp_doi_df["doi"].map(lambda x: get_assessment(url="https://doi.org/" + x))
comp_doi_df["assessment_result"] = comp_doi_df["assessment_result"].map(lambda x: get_metrics_and_scores(eval_result=x))
comp_doi_df.to_csv("assessed_data/comparison_doi_df_FES_assessment.csv")
comp_doi_df

Couldn't find program: 'false'


### Comparisons evaluation via DOI + visualizations

In [91]:
comp_doi_df = pd.read_csv("assessed_data/comparison_doi_df_FES_assessment.csv", index_col=0)
comp_doi_df["assessment_result"] = comp_doi_df["assessment_result"].map(lambda x: x.replace("\'", "\""))
comp_doi_df["assessment_result"] = comp_doi_df["assessment_result"].map(lambda x: json.loads(x))
comp_doi_df_assessed = expand_comp_assessment_result(comp_doi_df)
comp_doi_df_assessed

,comparison,doi,metric,score
0,http://orkg.org/orkg/resource/R140347,10.48366/r140347,F1A,1
1,http://orkg.org/orkg/resource/R140347,10.48366/r140347,F1B,1
2,http://orkg.org/orkg/resource/R140347,10.48366/r140347,F1B,0
3,http://orkg.org/orkg/resource/R140347,10.48366/r140347,F2A,1
4,http://orkg.org/orkg/resource/R140347,10.48366/r140347,F2B,1
...,...,...,...,...
6463,http://orkg.org/orkg/resource/R576876,10.48366/r576876,I2A,1
6464,http://orkg.org/orkg/resource/R576876,10.48366/r576876,I2B,0
6465,http://orkg.org/orkg/resource/R576876,10.48366/r576876,I3A,1
6466,http://orkg.org/orkg/resource/R576876,10.48366/r576876,R1.1,0


## Comparisons via ORKG URL

# Resources